## 实体链指比赛方案分享

### 1. **任务与难点介绍**
面向中文短文本的实体链指，简称 EL（Entity Linking），是NLP、知识图谱领域的基础任务之一，即对于给定的一个中文短文本（如搜索 Query、微博、对话内容、文章/视频/图片的标题等），EL将其中的实体与给定知识库中对应的实体进行关联。
此次任务的输入输出定义如下：
输入：中文短文本以及该短文本中的实体集合。
输出：输出文本此中文短文本的实体链指结果。每个结果包含：实体 mention、在中文短文本中的位置偏移、其在给定知识库中的 id，如果为 NIL 情况，需要再给出实体的上位概念类型。
传统的实体链指任务主要是针对长文档，长文档拥有在写的上下文信息能辅助实体的歧义消解并完成链指。相比之下，针对中文短文本的实体链指存在很大的挑战，主要原因如下：

（1）口语化严重，导致实体歧义消解困难；

（2）短文本上下文语境不丰富，须对上下文语境进行精准理解；

（3）相比英文，中文由于语言自身的特点，在短文本的链指问题上更有挑战。

### 2. **思路介绍**
可以将面向中文短文本的实体链指任务拆分为实体消歧与实体分类两个子任务，然后使用两个模型分别解决。针对实体消歧任务，沿用[1],[2]中的做法，将其转化为句对的二分类任务。针对实体分类任务，采用文本分类算法解决。

### 3. **数据处理方式**

#### 3.1 **候选实体获取**
采用字典匹配方式构建候选实体集合。字典的构造逻辑如下：

a.构建空字典D

b.遍历知识库，对每个mention，将其subject字段和alias字段中出现的实体作为key，subject_id作为value 

c.若key已出现在字典D中，将value添加到对应的list中;否则，新建一个空的list作为key在D中的值，然后将value添加进去 

按照上述方式得到字典D后，对每个实体，若其出现在字典中，取其在字典D中对应的list作为候选实体；否则，候选实体集合为空。 

#### 3.2 **实体链指数据处理**
对知识库中的每个实体，可以借用[1]的做法，将其实体属性拼接得到实体的描述文本。考虑到Type，摘要，义项描述等信息比较重要，本文按照Type，摘要，义项描述，其他信息的顺序进行拼接。其中Type字段在拼接前会先转成中文。
针对实体消歧任务，对短文本中出现的某个实体（本文称之为中心实体），按照[2]的做法，将短文本与中心实体对应的知识库中的实体描述文本作为正样本（标签为1），从候选集合中随机选取与正样本不同的实体描述文本作为负样本（标签为0）。由于各实体对应的候选实体集合大小不一，为了保证正负样本的均衡，可以采用随机负采样的方法来减少负样本的数量。为了充分利用负样本，本文使用了基于动态采样的方法，即每个epoch训练时重新采样负样本。

### 4. **模型设计**

#### 4.1 **实体消歧模型**
本文采用基于预训练语言模型的句对分类算法解决实体消歧任务。具体的，将短文本作为text_a，实体的描述文本作为text_b，将其按照[CLS]text_a[SEP]text_b[SEP]的格式输入到BERT[3]中，然后取[CLS]对应位置的向量作二分类。考虑到每个短文本中可能出现多个实体，在中心实体前后添加特殊符号###以标记中心实体的位置。模型示意图如下：

![](https://ai-studio-static-online.cdn.bcebos.com/72746dba9cc44c6086717bf08dfc2948dce21175dc754d6a82bc9e7485203180)

在模型推理阶段，从字典D中取出中心实体的候选集合S（若候选集合S为空时，将中心实体标记为NIL），得到每个候选实体的概率（预测标签为1的概率）。若其中的最大值大于某个阈值（可以作为一个超参数调节），则取概率最大的作为中心实体在知识库中的对应实体，否则将中心实体标记为NIL，然后使用实体分类模型对其进行分类。

#### 4.2 **实体分类模型**
类似地，本文采用基于预训练语言模型的文本分类算法解决实体分类任务。具体来说，将短文本作为text_a，将其按照[CLS]text_a[SEP]的格式输入到BERT中，然后取[CLS]对应位置的向量作多分类。在中心实体前后添加特殊符号###以标记中心实体的位置，这样模型能够利用实体的位置信息。模型示意图如下：

![](https://ai-studio-static-online.cdn.bcebos.com/6f114cf50f6f4bc59a49699967cb103f0052246e3c1240c18218ed9c5791bf45)

在训练阶段，本文使用中心实体不在知识库中的样本训练；在模型推理时，基于实体消歧模型的输出预测NIL的类别。考虑到训练集中包含大量非NIL实体（类别已知），为了充分利用这些信息，本文采用[1]中的两阶段训练的办法，即先在非NIL实体数据上微调预训练模型，然后在NIL实体数据上继续训练。为了提高模型的泛化能力，尝试了基于FGM[4]，PGD[5]的对抗训练策略来训练实体分类模型。

#### 4.3 **模型融合**：
本文使用简单的概率平均办法进行模型融合。对实体消歧模型，本文基于RoBERT-wwm-base[6]和ERNIE-1.0[7]总共训练了7个模型（不同的训练数据，随机种子等）。对实体分类模型，本文基于ERNIE-1.0训练了5个模型（不同的训练数据）。

### 5. **实验与结果分析**
实验基于百度提供的数据，训练集包含7万条数据，验证集包含1万条数据。A榜测试集包含1万条数据，B榜测试数据包含2.5万条数据。本次竞赛的数据从自然网页标题、多模标题、搜索query中抽取得到，通过人工众包标注，知识库实体重复率约5%，实体上位概念准确率95.27%，数据集标注准确率95.32%。该任务的知识库来自百度百科知识库。知识库中的每个实体都包含一个subject_id(知识库id)，一个subject名称，实体的别名，对应的概念类型，以及与此实体相关的一系列二元组< predicate，object>（<属性，属性值>）信息形式。知识库中每行代表知识库的一条记录（一个实体信息），每条记录为json数据格式。评估方式采用F1分值。特别注意，在计算评价指标时，对NIL实体的上位概念类型判断结果NIL_Type与实体的关联id等价处理。
实验参数设置如下：对ERNIE-1.0，学习率设置为5e-5，对RoBERT-wwm-base学习率取3e-5，epochs均为3，warmup比例设置为0.1。实体消歧模型采用动态负采样，每个中心实体采样的负样本数设置为2，batch size取64，最大序列长度为256。实体分类模型batch size为64，最大序列长度设置为72。

#### 5.1 **实体消歧模型结果**
下面从负采样的数目，是否使用动态负采样，阈值调整等方面进行实验，结果如下：


| 模型 | 验证集F1 | A榜F1 |
| -------- | -------- | -------- |
| ernie     | 0.857     | -     |
| ernie + 动态负采样     | 0.871     | 0.878     |
| ernie + 动态负采样 + 负采样2     | 0.872     | 0.880    |
| ernie + 动态负采样 + 负采样3     | 0.875    | 0.876    |
| ernie + 动态负采样 + 负采样2 + 阈值调整     | 0.878     | 0.882    |

从上表第一，第二行的结果可以看出，使用动态负采样优于静态负采样，因为动态负采样能更高效地利用负样本。从第二，第三，第四行的结果可以看出，当负样本个数取2时，模型效果最好。上面的实验结果与[2]中的结果基本一致。对比第三行与第五行，阈值调整能带来一定的性能提升。

#### 5.2 **实体分类模型结果**
| 模型  | 验证集F1 | A榜F1|
| -------- | -------- | -------- |
| 直接训练     | 0.870     | -    |
| 两阶段训练     | 0.878     | 0.882   |
| 两阶段训练 + FGM    | 0.880     | 0.883   |
| 两阶段训练 + PGD     | 0.879    | 0.883   |

从上表可以看出，两阶段训练能够提升模型的性能。在此基础上使用对抗训练也能带来较小的提升。综合考虑下，本文在训练实体消歧模型时使用动态负采样，采样数为2，并且添加了阈值的调整；在训练实体分类时采用两阶段训练加FGM的策略。

### 6. **总结**
本文将面向中文短文本的实体链指任务拆分为实体消歧与实体分类两个子任务。对实体消歧任务，采用基于预训练语言模型的句子对分类算法；对实体分类任务，使用基于预训练语言模型的文本分类模型。此外，本文还使用了一些技巧提升模型的性能，比如：对抗学习，阈值调整，模型融合等。在A榜测试集上取得0.8889的成绩，在B榜测试集上取得0.90981的成绩。本文的方案仍有地方需要改进，当前知识库的使用只是简单的将各属性连接，由于输入长度的限制，在输入到BERT后可能会有信息的丢失，可以设计更好的方案提取最重要的属性。 

最后说一下paddle的使用体验，个人感觉还是挺香的。paddle的动态图模式与torch的使用比较相似，熟悉torch的同学基本可以轻松入门。

### 7. **参考文献**
[1]吕荣荣, 王鹏程, 陈帅. 面向中文短文本的多因子融合实体链指研究
https://bj.bcebos.com/v1/conference/ccks2020/eval_paper/ccks2020_eval_paper_2_1.pdf

[2]潘春光, 王胜广, 罗志鹏. 知识增强的实体消歧与实体类别判断 
https://bj.bcebos.com/v1/conference/ccks2020/eval_paper/ccks2020_eval_paper_2_2.pdf

[3]Devlin J, Chang M W, Lee K, et al. Bert: Pre-training of deep bidirectional 
transformers for language understanding[J]. arXiv preprint arXiv:1810.04805, 2018.

[4]Goodfellow, Ian J, Shlens, Jonathon, and Szegedy, Christian. Explaining and harnessing adversarial examples.International Conference on Learning Representations(ICLR), 2015.

[5]Aleksander Madry, Aleksandar Makelov, Ludwig Schmidt,et al. Towards Deep Learning Models Resistant to Adversarial Attacks. https://arxiv.org/abs/1706.06083

[6]Cui Y, Che W, Liu T, et al. Pre-training with whole word masking for chinese bert[J]. 
arXiv preprint arXiv:1906.08101, 2019.

[7]Sun Y, Wang S, Li Y, et al. Ernie: Enhanced representation through knowledge 
integration[J]. arXiv preprint arXiv:1904.09223, 2019.

### **代码部分**

In [ ]:
## 环境配置：基于PaddlePaddle 1.8.4开发（python 3.7）， 使用单块V100（32G）训练

## 各个文件的作用：
### eval.py  官方提供的评估脚本
### post_matching.py 实体消歧模型后处理，对每个实体选取概率最大的一个作为kb_id（若小于一个阈值，则取NIL）（单模）
### main_nil.py 实体分类模型的推理代码，对实体消歧模型预测为NIL的实体预测其类别
### post_nil.py 实体分类模型的后处理代码，生成提交文件（单模）
### utils.py 定义各种训练，推理过程中需要的函数等
### main_matching.py 实体消歧模型推理代码
### train_matching.py 实体消歧模型训练代码
### post_matching_ens.py 实体消歧模型后处理，对每个实体选取概率最大的一个作为kb_id（若小于一个阈值，则取NIL）（多个模型）
### train_nil.py 实体分类模型训练代码
### post_nil_ens.py 实体分类模型的后处理代码，生成提交文件（多个模型）

In [ ]:
## 环境安装
!pip install paddle-ernie==0.0.4dev1

In [ ]:
## 训练实体消歧模型（基于ernie），需要20G左右显存，若是分配的16G会出现OOM，可以传入--bsz 16
!python work/train_matching.py --from_pretrained ernie-1.0 --use_lr_decay --save_dir work/tmp/dynamic_neg1_all --max_seqlen 256

In [ ]:
## 训练实体分类模型（基于ernie），两阶段训练，第一阶段利用非NIL实体，第二阶段使用第一阶段得到的模型在NIL上finetune
!python work/train_nil.py --from_pretrained ernie-1.0 --use_lr_decay --save_dir work/checkpoint/nil_preround --preround --epochs 2
!python work/train_nil.py --from_pretrained ernie-1.0 --use_lr_decay --save_dir work/checkpoint/nil_ft_preround --init_checkpoint work/checkpoint/nil_preround_debug.pdparams

In [ ]:
## 单模型推理代码(1个matching, 1个nil)
### 实体消歧模型推理
!python work/main_matching.py --from_pretrained ernie-1.0 --init_checkpoint work/checkpoint/dynamic_neg2_all.pdparams --save_path work/result/test_matching.pkl --max_seqlen 256 --use_test_data

### 实体消歧模型后处理
!python work/post_matching.py --use_test_data --thres 0.2

### 实体分类模型推理
!python work/main_nil.py --from_pretrained ernie-1.0 --init_checkpoint work/checkpoint/nil_ft_ad.pdparams --save_path work/result/test_nil.pkl --use_test_data

### 实体分类模型后处理
!python work/post_nil.py --use_test_data

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 